In [2]:
#from google.colab import drive
#mount your drive.  Complete Oauth to authenticate
#drive.mount('/content/gdrive')

In [3]:
#unzip image folder
#!unzip -uq "/content/gdrive/My Drive/jpegs.zip" -d "/content/gdrive/My Drive/"

In [59]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

# Train our network
def train_network(epochs, val_percent, train_batch_size, test_batch_size, eval_freq):

    # Training set values (same for all data)
    data_means = [0.6786, 0.6413, 0.6605]
    data_stds = [0.2012, 0.2080, 0.1997]

    transformations = transforms.Compose([
    #    transforms.Resize(255),
    #    transforms.CenterCrop(224),
        transforms.ToTensor(),  # Transforms channels from 0- 255 -> 0-1.
        transforms.Normalize(mean=data_means, std=data_stds)])

    #epochs = 10
    #val_percent = 0.2
    #train_batch_size = 25
    #test_batch_size = 25
    #eval_freq = 1

    #full_train_set = datasets.ImageFolder("/content/gdrive/My Drive/TRAIN", transform=transformations)
    full_train_set = datasets.ImageFolder("./jpegs/TRAIN", transform=transformations)
    #full_train_set, temp = torch.utils.data.random_split(full_train_set, [int(len(full_train_set) / 20), len(full_train_set) - int(len(full_train_set) / 20)])
    full_train_loader = torch.utils.data.DataLoader(full_train_set, batch_size=train_batch_size, shuffle=True)
    print("Full train set size: ", len(full_train_set))

    val_size = int(len(full_train_set)*val_percent)
    train_set, val_set = torch.utils.data.random_split(full_train_set, [len(full_train_set) - val_size, val_size])
    print("Train set size: ", len(train_set))
    print("Validation set size: ", len(val_set))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=train_batch_size, shuffle=True)

    #test_set = datasets.ImageFolder("/content/gdrive/My Drive/TEST", transform=transformations)
    test_set = datasets.ImageFolder("./jpegs/TEST", transform=transformations)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    print("Test set size: ", len(test_set))

    # Options: MOST GENERALIZED - 121, 169, 201, 161 - MOST ACCURATE
    # https://pytorch.org/hub/pytorch_vision_densenet/
    model = models.densenet161(pretrained=True)


    for param in model.parameters():
        param.requires_grad = False
    classifier_input = model.classifier.in_features
    num_labels = 4
    classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                               nn.ReLU(),
                               nn.Linear(1024, 512),
                               nn.ReLU(),
                               nn.Linear(512, num_labels),
                               nn.LogSoftmax(dim=1))


    model.classifier = classifier

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)

    # Can choose various loss functions to use:
    #criterion = nn.NLLLoss()
    #criterion = nn.CrossEntropyLoss()
    criterion = nn.MultiMarginLoss(margin=1.0)
    
    # Set the optimizer function using torch.optim as optim library
    optimizer = optim.Adam(model.classifier.parameters())

    for epoch in range(epochs):
        train_loss = 0
        val_loss = 0
        accuracy = 0

        # Training the model
        model.train()
    #    counter = 0
        for inputs, labels in train_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Clear optimizers
            optimizer.zero_grad()
            # Forward pass
            output = model.forward(inputs)
            # Loss
            loss = criterion(output, labels)
            # Calculate gradients (backpropogation)
            loss.backward()
            # Adjust parameters based on gradients
            optimizer.step()
            # Add the loss to the training set's rnning loss
            train_loss += loss.item()*inputs.size(0)

            # Print the progress of our training
    #        counter += 1
    #        if epoch == 0:
    #            print(counter, "/", len(train_loader))

        if (epoch % eval_freq == 0):

            # Evaluating the model
            model.eval()
    #        counter = 0
            # Tell torch not to calculate gradients
            with torch.no_grad():
                for inputs, labels in val_loader:
                    # Move to device
                    inputs, labels = inputs.to(device), labels.to(device)
                    # Forward pass
                    output = model.forward(inputs)
                    # Calculate Loss
                    valloss = criterion(output, labels)
                    # Add loss to the validation set's running loss
                    val_loss += valloss.item()*inputs.size(0)

                    # Since our model outputs a LogSoftmax, find the real
                    # percentages by reversing the log function
                    output = torch.exp(output)
                    # Get the top class of the output
                    top_p, top_class = output.topk(1, dim=1)
                    # See how many of the classes were correct?
                    equals = top_class == labels.view(*top_class.shape)
                    # Calculate the mean (get the accuracy for this batch)
                    # and add it to the running accuracy for this epoch
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                    # Print the progress of our evaluation
    #                counter += 1
    #                if epoch == 0:
    #                    print(counter, "/", len(val_loader))

            # Get the average loss for the entire epoch
            train_loss = train_loss/len(train_loader.dataset)
            val_loss = val_loss/len(val_loader.dataset)
            accuracy = accuracy/len(val_loader)
            # Print out the information
            print('Epoch: {} \tAccuracy: {:.6f} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} '.format(epoch+1, accuracy, train_loss, val_loss))



    full_train_loss = 0
    test_loss = 0
    test_accuracy = 0

    # Training the model one final time on the full dataset
    model.train()
    #counter = 0
    for inputs, labels in full_train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        full_train_loss += loss.item()*inputs.size(0)

        # Print the progress of our training
     #   counter += 1
     #   print(counter, "/", len(full_train_loader))

    # Saving the model
    torch.save(model, "./blood_model_e{}_v{}_b{}.py".format(epochs, val_percent, train_batch_size))

    model.eval()
    #counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            testloss = criterion(output, labels)
            # Add loss to the validation set's running loss
            test_loss += testloss.item()*inputs.size(0)

            # Since our model outputs a LogSoftmax, find the real
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            # Print the progress of our evaluation
    #        counter += 1
    #        print(counter, "/", len(test_loader))

    # Get the average loss for the entire fold
    full_train_loss = full_train_loss/len(full_train_loader.dataset)
    test_loss = test_loss/len(test_loader.dataset)
    test_accuracy = test_accuracy/len(test_loader)
    # Print out the information
    #print('Test Set Accuracy: ', test_accuracy)
    print('Final Results:\tAccuracy: {:.6f} \tTraining Loss: {:.6f} \tTesting Loss: {:.6f} \n'.format(test_accuracy, full_train_loss, test_loss))


# Process our image
def process_image(image_path):
    # Load Image
    img = Image.open(image_path)

    # Get the dimensions of the image
    width, height = img.size

    # Resize by keeping the aspect ratio, but changing the dimension
    # so the shortest size is 255px
    # img = img.resize((255, int(255*(height/width))) if width < height else (int(255*(width/height)), 255))

    # Get the dimensions of the new image size
    width, height = img.size

    # Set the coordinates to do a center crop of 224 x 224
    #left = (width - 224)/2
    #top = (height - 224)/2
    #right = (width + 224)/2
    #bottom = (height + 224)/2
    #img = img.crop((left, top, right, bottom))

    # Turn image into numpy array
    img = np.array(img)

    # Make the color channel dimension first instead of last
    img = img.transpose((2, 0, 1))

    # Make all values between 0 and 1
    img = img/255

    # Normalize based on the preset mean and standard deviation
    img[0] = (img[0] - data_means[0])/data_stds[0]
    img[1] = (img[1] - data_means[1])/data_stds[1]
    img[2] = (img[2] - data_means[2])/data_stds[2]

    # Add a fourth dimension to the beginning to indicate batch size
    img = img[np.newaxis,:]

    # Turn into a torch tensor
    image = torch.from_numpy(img)
    image = image.float()
    return image

# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model.forward(image)

    # Reverse the log function in our output
    output = torch.exp(output)

    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

# Show Image
def show_image(image):
    # Convert image to numpy
    image = image.numpy()

    # Un-normalize the image with avg std and mean
    image[0] = image[0] * 0.2030 + 0.6601

    # Print the image
    fig = plt.figure(figsize=(25, 4))
    plt.imshow(np.transpose(image[0], (1, 2, 0)))



In [ ]:
t_epochs = [5,10,25,50,100]
t_batches = [25,50,100,200]

for i in t_epochs:
    for j in t_batches:
        print('\nNumber of Epochs {} \tBatch Size: {}'.format(i,j))
        train_network(epochs=i, val_percent=0.2, train_batch_size=j, test_batch_size=j, eval_freq=1)

#train_network(epochs=3, val_percent=0.2, train_batch_size=25, test_batch_size=25, eval_freq=1)

#epochs = 10
#val_percent = 0.2
#train_batch_size = 25
#test_batch_size = 25
#eval_freq = 1


Number of Epochs 5 	Batch Size: 25
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487


In [41]:
'''print(len(full_train_set),len(full_train_set.targets))

# https://deeplizard.com/learn/video/0LhiS6yu2qQ
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

with torch.no_grad():
    full_train_preds = get_all_preds(model, full_train_loader)
    
preds_correct = get_num_correct(full_train_preds, full_train_set.targets)

print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(full_train_set))'''

9957 9957


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_conv2d_forward

In [42]:
'''
Run 1

epochs = 10
val_percent = 0.2
train_batch_size = 25
test_batch_size = 25
eval_freq = 1

criterion = nn.NLLLoss()



Epoch: 1 	Accuracy: 0.792156 	Training Loss: 0.866748 	Validation Loss: 0.507381 

Epoch: 2 	Accuracy: 0.828156 	Training Loss: 0.480419 	Validation Loss: 0.436654 

Epoch: 3 	Accuracy: 0.832219 	Training Loss: 0.411089 	Validation Loss: 0.421812 

Epoch: 4 	Accuracy: 0.882937 	Training Loss: 0.364378 	Validation Loss: 0.311798 

Epoch: 5 	Accuracy: 0.884000 	Training Loss: 0.334123 	Validation Loss: 0.299411 

Epoch: 6 	Accuracy: 0.839937 	Training Loss: 0.283902 	Validation Loss: 0.376899 

Epoch: 7 	Accuracy: 0.897500 	Training Loss: 0.245882 	Validation Loss: 0.265919 

Epoch: 8 	Accuracy: 0.845719 	Training Loss: 0.243905 	Validation Loss: 0.433223 

Epoch: 9 	Accuracy: 0.909000 	Training Loss: 0.221514 	Validation Loss: 0.226245 

Epoch: 10 	Accuracy: 0.908156 	Training Loss: 0.224361 	Validation Loss: 0.237138 


Test Set Accuracy:  0.609033337533474
Training Loss: 0.214690 	Testing Loss: 1.398789
'''

''